In [7]:
!pip install --upgrade jupyterlab
!pip install --upgrade jupyter
!pip install ipywidgets

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     |████████████████████████████████| 9.2 MB 7.1 MB/s eta 0:00:01
     |████████████████████████████████| 57 kB 98.0 MB/s  eta 0:00:01
     |████████████████████████████████| 377 kB 114.9 MB/s eta 0:00:01
     |████████████████████████████████| 65 kB 94.1 MB/s  eta 0:00:01
     |████████████████████████████████| 427 kB 129.8 MB/s eta 0:00:01
     |████████████████████████████████| 1.1 MB 84.9 MB/s eta 0:00:01
     |████████████████████████████████| 83 kB 65.8 MB/s  eta 0:00:01
     |████████████████████████████████| 84 kB 93.6 MB/s  eta 0:00:01
     |████████████████████████████████| 256 kB 121.9 MB/s eta 0:00:01
     |████████████████████████████████| 77 kB 102.9 MB/s eta 0:00:01
     |████████████████████████████████| 57 kB 118.6 MB/s eta 0:00:01
     |████████████████████████████████| 105 kB 112.0 MB/s eta 0:00:01
     |████████████████████████████████| 83 kB 66.2 MB/s  eta 0:00:01
     |█████████████████████

In [8]:

import os
import pandas as pd
import sys
#sys.path.append(0, os.path.abspath('ComParE2022_VecNet/src'))
sys.path.append('../../src')

import config ,config_pytorch
#from evaluate import get_results
import numpy as np

# Troubleshooting and visualisation
# import IPython.display as ipd

# humbug lib imports
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
#from PyTorch import config_pytorch
from datetime import datetime
import math
import pickle

from torch.utils.data import TensorDataset, DataLoader
import torch.nn.functional as F
import torch.nn as nn
import torch
import torch.optim as optim
import numpy as np
from sklearn.metrics import accuracy_score
from datetime import datetime
import time

import matplotlib
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import precision_recall_curve, plot_precision_recall_curve
from sklearn.metrics import average_precision_score
import sys

from tqdm.notebook import tqdm
# additional pytorch tools
import random
import torchaudio
import torchaudio.transforms as AT
import torchvision.transforms as VT
from torch.cuda.amp import autocast, GradScaler
from timm.scheduler.cosine_lr import CosineLRScheduler
import timm
import timm.optim
from timm.loss import BinaryCrossEntropy
from timm.utils import NativeScaler
from timm.models import model_parameters
from glob import glob
## nnAudio
from nnAudio import features , Spectrogram
from torch.utils.data.sampler import SubsetRandomSampler
from torch.utils.data import Dataset, DataLoader
import argparse


/opt/conda/lib/python3.8/site-packages/nnAudio/Spectrogram.py:4: Warning: importing Spectrogram subpackage will be deprecated soon. You should import the feature extractor from the feature subpackage. See actual documentation.
  warnings.warn(


In [9]:
num_calibration_batches = 32


In [10]:
class MyModel(nn.Module):
    def __init__(self, model_name, image_size = 224):
        super().__init__()
        # num_classes=0 removes the pretrained head
        self.backbone = timm.create_model(model_name,
                        pretrained=True, num_classes=2, in_chans=1, 
                        drop_path_rate=0.2, global_pool='max',
                        drop_rate=0.25)
        #####  This section is model specific
        #### It freezes some fo the layers by name
        #### you'll have to inspect the model to see the names
                #### end layer freezing
        self.out = nn.Linear(self.backbone.num_features, 1)
        self.sizer = VT.Resize((image_size,image_size),antialias = True)
        self.spec_layer = features.STFT(n_fft=int(config.NFFT), freq_bins=None, hop_length=int(config.n_hop),
                              window='hann', freq_scale='linear', center=True, pad_mode='reflect',
                           sr=config.rate, output_format="Magnitude", trainable=False,verbose = False).to('cuda')
        self.batch_norm = nn.BatchNorm2d(num_features= 1)
        #self.augment_layer = augment_audio(trainable = True, sample_rate = config.rate)
        self.quant = torch.ao.quantization.QuantStub()
        self.dequant = torch.ao.quantization.DeQuantStub()
        
    def forward(self, x,train = True):
        # first compute spectrogram
        x = self.quant(x)
        spec_gram = self.spec_layer(x)
        #print("post spec gram shape = ",spec_gram.shape)
        spec_gram = self.batch_norm(spec_gram.unsqueeze(dim = 1))
        #print("post norm shape = ",spec_gram.shape)
        spec_gram_nan_check = torch.isnan(spec_gram).any().item()
        assert not (spec_gram_nan_check) ,"Tensor contains NaN values after spec gram creation."
        
        with torch.no_grad():
            if train == True:
                #generate a random number and if condition is met apply aug
                ta_transformations_rndm_choice = VT.RandomChoice([AT.FrequencyMasking(freq_mask_param=100),AT.TimeMasking(time_mask_param=50)], p=[.4, .4])
                ta_transformations_rndm_apply = VT.RandomApply([AT.FrequencyMasking(freq_mask_param=50),AT.TimeMasking(time_mask_param=25)],p = .15)
                spec_gram = ta_transformations_rndm_choice(spec_gram)
                spec_gram = ta_transformations_rndm_apply(spec_gram)
                spec_gram_nan_check = torch.isnan(spec_gram).any().item()
                assert not (spec_gram_nan_check) ,"Tensor contains NaN values after augmentations  "
                
                
            
        
        x = self.sizer(spec_gram.squeeze(dim = 1))
        #print("post sizer shape = ",x.shape)
        x = x.unsqueeze(dim = 1)
        #print("post unsqueeze shape = ",x.shape)
        
        # then repeat channels
        del spec_gram,spec_gram_nan_check
        if DEBUG:
            print("Final shape that goes to backbone = " + str(x.shape))
                
        x = self.backbone(x)
        backbone_op_nan_check = torch.isnan(x).any().item()
        assert not (backbone_op_nan_check) ,"Tensor contains NaN values in the backbone OP "
        #print("x shape = " + str(x.shape))
        #print("x = " +str(x))
        #pred = nn.Softmax(x)
        x = self.dequant(x)
        pred = x
        #print(np.argmax(pred.detach().cpu().numpy()))
        #print(pred)
        output = {"prediction": pred }
        #print(output)
        del x , backbone_op_nan_check
        return output


In [11]:
def load_model(model_file):
    model =MyModel('convnext_small',384)
    state_dict = torch.load(model_file)
    model.load_state_dict(state_dict)
    model.to('cpu')
    return model

In [15]:
import os
model_file = os.path.join("..","..","models","model_med12_2023_10_30_08_18_32.pth")

In [16]:
print(model_file)

../../models/model_med12_2023_10_30_08_18_32.pth


In [18]:
myModel = load_model(model_file).to('cpu')
#print_model_size(myModel)

In [19]:
num_params = sum(p.numel() for p in myModel.parameters())

# Convert the number of parameters to megabytes
model_size_mb = num_params * 4 / (1024 ** 2)  
print(model_size_mb)

188.65175247192383


In [21]:
model_int8_384 = torch.quantization.quantize_dynamic(myModel,  # the original model
    {torch.nn.Linear, torch.nn.Conv2d, torch.nn.GELU, torch.nn.BatchNorm2d}, 
    dtype=torch.qint8) 

#num_params = sum(p.numel() for p in model_int8.parameters())

# Convert the number of parameters to megabytes
#model_size_mb = num_params * 4 / (1024 ** 2)  
print(model_size_mb)

188.65175247192383


In [18]:
!pwd

/dli/task/ComParE2022_VecNet/notebooks/DK


In [23]:
chk_pt_name = "model_int8_384.pth"
torch.save(model_int8_384.state_dict(), chk_pt_name)
